In [7]:
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder, util

In [8]:
# Charger les données JSON
json_file_path = "linked_files.json"
with open(json_file_path, "r", encoding="utf-8") as file:
    json_data = json.load(file)

In [9]:
# Extraire les titres et le contenu technique
documents = []
for entry in json_data:
    title = entry["title"]
    content = entry["content"]
    # Extraire uniquement la partie "Technique" du contenu
    if "# Technique" in content:
        technique_section = content.split("# Technique")[1].split("#")[0].strip()
    else:
        technique_section = ""
    # Combiner le titre et le contenu technique
    combined_text = f"{title} {technique_section}"
    documents.append(combined_text)

In [20]:
# Initialiser le modèle SBERT et le Cross-Encoder
sbert_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-electra-base")
#cross-encoder/ms-marco-MiniLM-L-12-v2
#cross-encoder/ms-marco-MiniLM-L-6-v2


# Calculer les embeddings des documents
embeddings = sbert_model.encode(documents, convert_to_tensor=True)

# Indexer avec FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
faiss.normalize_L2(embeddings.numpy())
index.add(embeddings.numpy())

In [21]:
# Fonction de recherche
def search(query, top_k=10):
    # Vectoriser la requête
    query_embedding = sbert_model.encode([query], convert_to_tensor=True)
    faiss.normalize_L2(query_embedding.numpy())
    
    # Rechercher les documents les plus proches avec FAISS
    _, faiss_results = index.search(query_embedding.numpy(), top_k)
    faiss_scores = [(documents[i], json_data[i]) for i in faiss_results[0]]
    
    # Réordonner les résultats avec le Cross-Encoder
    cross_scores = cross_encoder.predict([(query, doc[0]) for doc in faiss_scores])
    ranked_results = [x for _, x in sorted(zip(cross_scores, faiss_scores), reverse=True)]
    
    return ranked_results[:top_k]

In [23]:
# Exemple de recherche
query = "Comment isoler ma maison"
results = search(query)
for doc, metadata in results:
    print(f"Titre: {metadata['title']}")
    print(f"Code: {metadata['identifier']}")
    print("---")

Titre: Isolation des séchoirs
Code: 1006
---
Titre: Isolation d'un plancher
Code: 252
---
Titre: Isolation des combles ou de toiture
Code: 250
---
Titre: Panneaux d'isolation sous vide
Code: 1575
---
Titre: Sur isolation du réseau de chaleur
Code: 1638
---
Titre: Amélioration de l'isolation des bâtiments
Code: 97
---
Titre: Isolation des murs par l'extérieur
Code: 254
---
Titre: Isolation des équipements qui ne le sont pas
Code: 333
---
Titre: Isolation des toitures terrasses et couvertures de faible pente
Code: 255
---
Titre: Isolation des murs par l'intérieur
Code: 251
---
